In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import functions as func
import csv
from scipy import stats
from scipy.stats import zscore
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
import scipy.stats as st
from statsmodels.stats.proportion import proportions_ztest
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week5\Project\Data\Raw\df_final_demo.txt'
df_f_dem_original = pd.read_csv(file_path)
df_f_dem = df_f_dem_original.copy()

file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week5\Project\Data\Raw\df_final_experiment_clients.txt'
df_f_ex_original = pd.read_csv(file_path)
df_f_ex = df_f_ex_original.copy()

file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week5\Project\Data\Raw\df_final_web_data_pt_1.txt'
df_f_we1_original = pd.read_csv(file_path)
df_f_we1 = df_f_we1_original.copy()
df_f_we1

file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week5\Project\Data\Raw\df_final_web_data_pt_2.txt'
df_f_we2_original = pd.read_csv(file_path)
df_f_we2 = df_f_we2_original.copy()

concatenated_df = pd.concat([df_f_we1, df_f_we2], ignore_index=True)
df_f_we = concatenated_df.sort_values(by=['client_id', 'date_time'])
df_f_we['date_time'] = pd.to_datetime(df_f_we['date_time'])

df_f_we.head(10)

,client_id,visitor_id,visit_id,process_step,date_time
285515,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36
285514,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45
285513,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31
285512,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05
285511,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09
628456,336,64757908_3400128256,649044751_80905125055_554468,start,2017-06-01 07:26:55
628331,336,64757908_3400128256,649044751_80905125055_554468,start,2017-06-01 07:42:43
442857,546,475037402_89828530214,731811517_9330176838_94847,start,2017-06-17 10:03:29
442856,546,475037402_89828530214,731811517_9330176838_94847,step_1,2017-06-17 10:03:39
442855,546,475037402_89828530214,731811517_9330176838_94847,step_2,2017-06-17 10:03:52


In [3]:
conditions = [
    df_f_we['process_step'] == 'start',
    df_f_we['process_step'] == 'step_1',
    df_f_we['process_step'] == 'step_2',
    df_f_we['process_step'] == 'step_3',
    df_f_we['process_step'] == 'confirm'
]
values = [0, 1, 2, 3, 4]

df_f_we['step_value'] = np.select(conditions, values, default=np.nan).astype(int)
df_f_we.head(10)

,client_id,visitor_id,visit_id,process_step,date_time,step_value
285515,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36,0
285514,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45,1
285513,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31,2
285512,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05,3
285511,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09,4
628456,336,64757908_3400128256,649044751_80905125055_554468,start,2017-06-01 07:26:55,0
628331,336,64757908_3400128256,649044751_80905125055_554468,start,2017-06-01 07:42:43,0
442857,546,475037402_89828530214,731811517_9330176838_94847,start,2017-06-17 10:03:29,0
442856,546,475037402_89828530214,731811517_9330176838_94847,step_1,2017-06-17 10:03:39,1
442855,546,475037402_89828530214,731811517_9330176838_94847,step_2,2017-06-17 10:03:52,2


In [4]:
dem_ex_merge = pd.merge(df_f_dem, df_f_ex, on='client_id', how='inner')

## Creating df for test group

In [5]:
filtered_dem_ex_merge_drop = dem_ex_merge[dem_ex_merge['Variation'] == 'Test']
client_ids = filtered_dem_ex_merge_drop['client_id'].unique()
filtered_df_f_we = df_f_we[df_f_we['client_id'].isin(client_ids)]
test_df_time = pd.merge(filtered_dem_ex_merge_drop, filtered_df_f_we, on='client_id')
test_df_time_sorted = test_df_time.sort_values(by=['client_id', 'date_time'])
test_df_time_sorted['visit_id_numbers'] = test_df_time_sorted['visit_id'].str.split('_').str[-1].astype(int)

In [6]:
client_ids_test_reached_confirmation = np.unique(test_df_time_sorted[test_df_time_sorted['process_step'] == "confirm"]['client_id'].values)
test_finish_df = test_df_time_sorted[ test_df_time_sorted['client_id'].isin(client_ids_test_reached_confirmation) ]

test_finish_df

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,visitor_id,visit_id,process_step,date_time,step_value,visit_id_numbers
141353,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,0,716659
141354,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,1,716659
141355,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,2,716659
141356,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,3,716659
141357,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,4,716659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171552,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,start,2017-05-08 16:08:25,0,814203
171553,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,step_1,2017-05-08 16:08:30,1,814203
171554,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,step_2,2017-05-08 16:08:40,2,814203
171555,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,step_3,2017-05-08 16:09:19,3,814203


## Calculating time differences for Test group.

Calling on function that calculates the total time from start to last confirmation in one session

In [82]:
test_total_times = func.calculate_total_times(test_finish_df)

In [78]:
total_time_seconds = sum(test_total_times, timedelta()) 
average_time = total_time_seconds / len(test_total_times)
average_time_seconds_test = average_time.total_seconds() 

print("Average time:", average_time_seconds, "seconds")

Average time: 333.058985 seconds


Calling on function that calculates time difference step by step

In [9]:
test_step_1_time_diff = func.calculate_time_differences_step(test_finish_df, 0, 1)
test_avg_time_diff_step1 = np.mean(test_step_1_time_diff)
test_avg_time_diff_step1

30.57463960922918

In [10]:
test_step_2_time_diff = func.calculate_time_differences_step(test_finish_df, 1, 2)
test_avg_time_diff_step2 = np.mean(test_step_2_time_diff)
test_avg_time_diff_step2

36.361507989495706

In [11]:
test_step_3_time_diff = func.calculate_time_differences_step(test_finish_df, 2, 3)
test_avg_time_diff_step3 = np.mean(test_step_3_time_diff)
test_avg_time_diff_step3

86.60626831804859

In [12]:
test_step_4_time_diff = func.calculate_time_differences_step(test_finish_df, 3, 4)
test_avg_time_diff_step4 = np.mean(test_step_4_time_diff)
test_avg_time_diff_step4

104.83331518187758

## Creating df control group 

In [13]:
filtered_dem_ex_merge_drop = dem_ex_merge[dem_ex_merge['Variation'] == 'Control']
client_ids = filtered_dem_ex_merge_drop['client_id'].unique()
filtered_df_f_we = df_f_we[df_f_we['client_id'].isin(client_ids)]
control_df_time = pd.merge(filtered_dem_ex_merge_drop, filtered_df_f_we, on='client_id')
control_df_time_sorted = control_df_time.sort_values(by=['client_id', 'date_time'])
control_df_time_sorted['visit_id_numbers'] = control_df_time_sorted['visit_id'].str.split('_').str[-1].astype(int)

In [14]:
client_ids_control_reached_confirmation = np.unique(control_df_time_sorted[control_df_time_sorted['process_step'] == "confirm"]['client_id'].values)
control_finish_df = control_df_time_sorted[ control_df_time_sorted['client_id'].isin(client_ids_control_reached_confirmation) ]
control_finish_df

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,visitor_id,visit_id,process_step,date_time,step_value,visit_id_numbers
127373,1195,21.0,262.0,54.5,M,2.0,28457.96,2.0,5.0,Control,766842522_69992551638,393817425_39015278493_996341,start,2017-04-05 20:15:26,0,996341
127374,1195,21.0,262.0,54.5,M,2.0,28457.96,2.0,5.0,Control,766842522_69992551638,393817425_39015278493_996341,step_1,2017-04-05 20:15:59,1,996341
127375,1195,21.0,262.0,54.5,M,2.0,28457.96,2.0,5.0,Control,766842522_69992551638,393817425_39015278493_996341,step_2,2017-04-05 20:17:37,2,996341
127376,1195,21.0,262.0,54.5,M,2.0,28457.96,2.0,5.0,Control,766842522_69992551638,393817425_39015278493_996341,step_3,2017-04-05 20:18:08,3,996341
127377,1195,21.0,262.0,54.5,M,2.0,28457.96,2.0,5.0,Control,766842522_69992551638,393817425_39015278493_996341,confirm,2017-04-05 20:19:31,4,996341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62790,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:30:11,2,783154
62791,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:30:25,3,783154
62792,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:37:28,3,783154
62793,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,confirm,2017-03-29 15:37:28,4,783154


<span style="font-size:20px;">Calculating time differences for Control group.</span>

Calling on function that calculates the total time from start to last confirmation in one session

In [15]:
control_total_times = func.calculate_total_times(control_finish_df)

In [86]:
control_total_seconds = [time_delta.total_seconds() for time_delta in control_total_times]

In [79]:
total_time_seconds = sum(control_total_times, timedelta()) 
average_time = total_time_seconds / len(control_total_times)
average_time_seconds_control = average_time.total_seconds() 

print("Average time:", average_time_seconds, "seconds")

Average time: 333.058985 seconds


In [17]:
control_step_1_time_diff = func.calculate_time_differences_step(control_finish_df, 0, 1)
control_avg_time_diff_step1 = np.mean(control_step_1_time_diff)
control_avg_time_diff_step1

36.79552359732631

In [18]:
control_step_2_time_diff = func.calculate_time_differences_step(control_finish_df, 1, 2)
control_avg_time_diff_step2 = np.mean(control_step_2_time_diff)
control_avg_time_diff_step2

32.939302389453445

In [19]:
control_step_3_time_diff = func.calculate_time_differences_step(control_finish_df, 2, 3)
control_avg_time_diff_step3 = np.mean(control_step_3_time_diff)
control_avg_time_diff_step3

86.19777516311905

In [20]:
control_step_4_time_diff = func.calculate_time_differences_step(control_finish_df, 3, 4)
control_avg_time_diff_step4 = np.mean(control_step_4_time_diff)
control_avg_time_diff_step4

127.14917560847945

In [90]:
test_total_time = average_time_seconds_test
test_time_step1 = test_avg_time_diff_step1
test_time_step2 = test_avg_time_diff_step2
test_time_step3 = test_avg_time_diff_step3
test_time_step4 = test_avg_time_diff_step4

control_total_time = average_time_seconds_control
control_time_step1 = control_avg_time_diff_step1
control_time_step2 = control_avg_time_diff_step2
control_time_step3 = control_avg_time_diff_step3
control_time_step4 = control_avg_time_diff_step4

# Write data to CSV file
csv_file_path = 'time_data.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Group', 'Metric', 'Time'])
    writer.writerow(['Test', 'Total Time Till Completion', test_total_time])
    writer.writerow(['Test', 'Time Step 1', test_time_step1])
    writer.writerow(['Test', 'Time Step 2', test_time_step2])
    writer.writerow(['Test', 'Time Step 3', test_time_step3])
    writer.writerow(['Test', 'Time Step 4', test_time_step4])
    writer.writerow(['Control', 'Total Time Till Completion', control_total_time])
    writer.writerow(['Control', 'Time Step 1', control_time_step1])
    writer.writerow(['Control', 'Time Step 2', control_time_step2])
    writer.writerow(['Control', 'Time Step 3', control_time_step3])
    writer.writerow(['Control', 'Time Step 4', control_time_step4])

print("CSV file created successfully:", csv_file_path)


CSV file created successfully: time_data.csv


<span style="font-size:20px;">Checking for error rate in Test group.</span>


In [21]:
def check_next_step_errors(df):
    error_count = 0
    for client_id, group in df.groupby('visit_id_numbers'):
        for i in range(len(group) - 1):
            if group.iloc[i]['step_value'] > group.iloc[i + 1]['step_value']:
                error_count += 1
                break
    return error_count

test_error = check_next_step_errors(test_df_time_sorted)
test_total_session = test_df_time_sorted['visit_id_numbers'].nunique()
test_error_rate = round(((test_error / test_total_session)*100), 2)

print(f"In the test group there are {test_error} errors out of {test_total_session} sessions.")
print(f"The ratio is {test_error_rate} %")

In the test group there are 10012 errors out of 37136 sessions.
The ratio is 26.96 %


<span style="font-size:20px;">Checking for error rate in Control group.</span>


In [22]:
def check_next_step_errors(df):
    error_count = 0
    for client_id, group in df.groupby('visit_id_numbers'):
        for i in range(len(group) - 1):
            if group.iloc[i]['step_value'] > group.iloc[i + 1]['step_value']:
                error_count += 1
                break
    return error_count

control_error = check_next_step_errors(control_df_time_sorted)
control_total_session = control_df_time_sorted['visit_id_numbers'].nunique()
control_error_rate = round(((control_error / control_total_session)*100), 2)

print(f"In the test group there are {control_error} errors out of {control_total_session} sessions.")
print(f"The ratio is {control_error_rate} %")

In the test group there are 6581 errors out of 32189 sessions.
The ratio is 20.44 %


In [111]:
data = {
    'Group': ['Test', 'Control'],
    'Errors': [test_error, control_error],
    'Sessions': [test_total_session, control_total_session],
    'Error Rate': [test_error_rate, control_error_rate]
}
error_df = pd.DataFrame(data)
error_df.to_csv('error_rates.csv', index=False)

print("CSV file created successfully.")

CSV file created successfully.


In [27]:
dem_ex_merge['Variation'].unique()

array(['Test', 'Control', nan], dtype=object)

In [28]:
nan_count = dem_ex_merge['Variation'].isna().sum()
nan_count 

20109

In [32]:
dem_ex_merge_drop = dem_ex_merge.dropna(subset=['Variation'])
dem_ex_merge_drop

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0,Test
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0,Control
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0,Test
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0,Test
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0,Control
...,...,...,...,...,...,...,...,...,...,...
50495,1780858,21.0,262.0,68.5,M,3.0,372100.59,6.0,9.0,Test
50496,6967120,21.0,260.0,68.5,M,3.0,4279873.38,6.0,9.0,Control
50497,5826160,20.0,249.0,56.5,F,2.0,44837.16,2.0,5.0,Test
50498,8739285,19.0,229.0,69.5,F,2.0,44994.24,1.0,4.0,Test


## Hypothesis Testing: Analysis of Variations in average age between Test and Control Groups


**H0:** The average age of clients engaging with the new process is not significantly greater than or equal to the average age of those engaging with the old process.

**H1:** The average age of clients engaging with the new process is significantly greater than the average age of those engaging with the old process.


In [35]:
new_age_bins = [0, 30, 50, 70, 90, float('inf')]
new_age_labels = ['0-29', '30-49', '50-69', '70-89', '90+']
dem_ex_merge_age_grouped = dem_ex_merge_drop.copy()
dem_ex_merge_age_grouped.dropna(subset=['clnt_age'], inplace=True)
dem_ex_merge_age_grouped['age_group'] = pd.cut(dem_ex_merge_age_grouped['clnt_age'], bins=new_age_bins, labels=new_age_labels, right=False)

age_counts_per_test = dem_ex_merge_age_grouped.groupby('age_group')['Variation'].value_counts()
age_counts_per_test_df = age_counts_per_test.reset_index(name='count')
print(age_counts_per_test_df)

  age_group Variation  count
0      0-29      Test   4596
1      0-29   Control   3935
2     30-49      Test   9898
3     30-49   Control   8442
4     50-69      Test  10672
5     50-69   Control   9513
6     70-89      Test   1770
7     70-89   Control   1618
8       90+      Test     25
9       90+   Control     18


In [36]:
age_test = dem_ex_merge_age_grouped[dem_ex_merge_age_grouped['Variation'] == 'Test']['clnt_age']
age_control = dem_ex_merge_age_grouped[dem_ex_merge_age_grouped['Variation'] == 'Control']['clnt_age']


t_statistic, p_value = ttest_ind(age_test, age_control, alternative='greater')

alpha = 0.05

print("One-Sided T-Test Results:")
print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < alpha:
    print("Reject the null hypothesis: The average age of clients engaging with the new process is significantly greater than those engaging with the old process.")
else:
    print("Fail to reject the null hypothesis: There is not enough evidence to suggest that the average age of clients engaging with the new process is significantly greater than those engaging with the old process.")

One-Sided T-Test Results:
T-statistic: -2.4161
P-value: 0.9922
Fail to reject the null hypothesis: There is not enough evidence to suggest that the average age of clients engaging with the new process is significantly greater than those engaging with the old process.


## Hypothesis Testing: Analysis of Variations in Tenure between Test and Control Groups


**H0:** The average tenure of clients engaging with the new process is not significantly greater than or equal to the average age of those engaging with the old process.

**H1:** The average tenure of clients engaging with the new process is significantly greater than or equal to the average age of those engaging with the old process.

In [38]:
tenure_test = dem_ex_merge_age_grouped[dem_ex_merge_age_grouped['Variation'] == 'Test']['clnt_tenure_mnth']
tenure_control = dem_ex_merge_age_grouped[dem_ex_merge_age_grouped['Variation'] == 'Control']['clnt_tenure_mnth']


t_statistic, p_value = ttest_ind(tenure_test, tenure_control, alternative='greater')

alpha = 0.05

print("One-Sided T-Test Results:")
print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < alpha:
    print("Reject the null hypothesis: The average tenure of clients engaging with the new process is significantly greater than those engaging with the old process.")
else:
    print("Fail to reject the null hypothesis: There is not enough evidence to suggest that the average tenure of clients engaging with the new process is significantly greater than those engaging with the old process.")

One-Sided T-Test Results:
T-statistic: -1.6540
P-value: 0.9509
Fail to reject the null hypothesis: There is not enough evidence to suggest that the average tenure of clients engaging with the new process is significantly greater than those engaging with the old process.


## Hypothesis Testing: Analysis of Variations in Gender between Test and Control Groups


**H0:** The amount of males engaging with the new process is not significantly greater than or equal to the average age of those engaging with the old process.

**H1:** The amount of males engaging with the new process is not significantly greater than or equal to the average age of those engaging with the old process.

In [39]:
gender_df = dem_ex_merge_drop[dem_ex_merge_drop['gendr'].isin(['F', 'M'])].copy()
gender_df['gendr'].unique()

array(['M', 'F'], dtype=object)

In [40]:
gender_test = gender_df[gender_df['Variation'] == 'Test']['gendr']
gendr_control = gender_df[gender_df['Variation'] == 'Control']['gendr']


contingency_table = pd.crosstab(gender_df['Variation'], gender_df['gendr'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

alpha = 0.05

print("Chi-Square Test Results:")
print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < alpha:
    print("Reject the null hypothesis: There is a significant association between gender and group.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between gender and group.")

Chi-Square Test Results:
Chi-square statistic: 1.3238
P-value: 0.2499
Fail to reject the null hypothesis: There is no significant association between gender and group.


In [41]:
common_values = dem_ex_merge_age_grouped['client_id'].isin(max_step_reached.index).value_counts()

print("Common values:", common_values)

Common values: client_id
True    50487
Name: count, dtype: int64


## Statistical Analysis of Completion Rate and Cost-Effectiveness Threshold


**H0:** There is no difference in completion rates between the Test and Control groups.

**H1:** There is a difference in completion rates between the Test and Control groups.


In [46]:
step_ranking = {'start': 1, 'step_1': 2, 'step_2': 3, 'step_3': 4, 'confirm': 5}
df_f_we_process = df_f_we.copy()
df_f_we_process['step_rank'] = df_f_we_process['process_step'].map(step_ranking)
max_step_reached = df_f_we_process.groupby('client_id')['step_rank'].max()

In [47]:
series_process_df = max_step_reached.to_frame(name='step_reached').reset_index()
step_reached_df = pd.merge(dem_ex_merge_age_grouped, series_process_df, on='client_id')

In [48]:
total_test = step_reached_df[step_reached_df['Variation'] == 'Test'].shape[0]
total_control = step_reached_df[step_reached_df['Variation'] == 'Control'].shape[0]

total_finished_test = step_reached_df[(step_reached_df['step_reached'] == 5) & (step_reached_df['Variation'] == 'Test')].shape[0]
total_finished_control = step_reached_df[(step_reached_df['step_reached'] == 5) & (step_reached_df['Variation'] == 'Control')].shape[0]

stat, p_value = proportions_ztest([total_finished_test, total_finished_control],
                                   [total_test, total_control])
alpha = 0.05

print(f"Z-score: {stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in completion rates between Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in completion rates between Test and Control groups.")

Z-score: 8.8927
P-value: 0.0000
Reject the null hypothesis: There is a significant difference in completion rates between Test and Control groups.


In [49]:
step5_counts_finish = step_reached_df[step_reached_df['step_reached'] == 5]['Variation'].value_counts()
step5_counts_finish
step_reached_df

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,age_group,step_reached
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0,Test,50-69,5
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0,Control,50-69,5
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0,Test,30-49,4
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0,Test,30-49,1
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0,Control,30-49,1
...,...,...,...,...,...,...,...,...,...,...,...,...
50482,1780858,21.0,262.0,68.5,M,3.0,372100.59,6.0,9.0,Test,50-69,5
50483,6967120,21.0,260.0,68.5,M,3.0,4279873.38,6.0,9.0,Control,50-69,5
50484,5826160,20.0,249.0,56.5,F,2.0,44837.16,2.0,5.0,Test,50-69,5
50485,8739285,19.0,229.0,69.5,F,2.0,44994.24,1.0,4.0,Test,50-69,5


In [63]:
step5_clients_df = step_reached_df[step_reached_df['step_reached'] == 5]
step5_counts = step5_clients_df['Variation'].value_counts()
total_clients = step_reached_df['Variation'].value_counts()
step5_percentage = (step5_counts / total_clients) * 100

print("Percentage of clients who reached step 5:")
print(step5_percentage)

Percentage of clients who reached step 5:
Variation
Test       69.292682
Control    65.578509
Name: count, dtype: float64


In [105]:
completion_df = pd.concat([total_clients, step5_counts, step5_percentage], axis=1)
completion_df.index = ['Test', 'Control']
completion_df

completion_df.reset_index(inplace=True)

completion_df.columns = ['Variation', 'Total clients', 'Complete all steps', 'Percentage']
completion_df

,Variation,Total clients,Complete all steps,Percentage
0,Test,26961,18682,69.292682
1,Control,23526,15428,65.578509


In [107]:
completion_df.to_csv('completion_group.csv', index=False)

print("CSV file created successfully.")

CSV file created successfully.


In [109]:
df_completion_rate = pd.DataFrame({'Variation': ['Test', 'Control'],
                   'Total clients': total_clients.values,
                   'Complete all steps': step5_counts.values,
                   'Percentage': step5_percentage.values})

# Export DataFrame to CSV
df_completion_rate.to_csv('merged_data_for_tableau.csv', index=False)

print("CSV file created successfully.")

CSV file created successfully.


In [64]:
percentage_diff = round(((step5_percentage.iloc[0] - step5_percentage.iloc[1]) / step5_percentage.iloc[1]) * 100 , 2)
print("Percentage difference between test and control:")
print(percentage_diff, '%')

Percentage difference between test and control:
5.66 %


## Grouping by balance, tenure and age

In [52]:
balance_bins = [0, 10000, 50000, 100000, 500000, 1000000, float('inf')]
balance_labels = ['0-10k', '10k-50k', '50k-100k', '100k-500k', '500k-1M', '1M+']

dem_ex_merge_balance_grouped = dem_ex_merge_drop.copy()

dem_ex_merge_balance_grouped.loc[:, 'balance_group'] = pd.cut(dem_ex_merge_balance_grouped['bal'], bins=balance_bins, labels=balance_labels, right=False)

balance_counts_per_test = dem_ex_merge_balance_grouped.groupby('balance_group')['Variation'].value_counts()
balance_counts_per_test_df = balance_counts_per_test.reset_index(name='count')
print(balance_counts_per_test_df)

   balance_group Variation  count
0          0-10k   Control      0
1          0-10k      Test      0
2        10k-50k      Test  10000
3        10k-50k   Control   8545
4       50k-100k      Test   7639
5       50k-100k   Control   6825
6      100k-500k      Test   7876
7      100k-500k   Control   6888
8        500k-1M      Test   1016
9        500k-1M   Control    871
10           1M+      Test    430
11           1M+   Control    398


In [95]:
csv_file_path = 'balance_group.csv'
balance_counts_per_test_df.to_csv(csv_file_path, index=False)  # Set index=False to exclude row numbers in the CSV file

print("DataFrame saved to CSV file:", csv_file_path)

DataFrame saved to CSV file: balance_group.csv


In [93]:
tenure_bins = [0, 5, 10, 15, 20, 25, float('inf')]
tenure_labels = ['0-5', '6-10', '11-15', '16-20', '21-25', '26+']
dem_ex_merge_tenure_grouped = dem_ex_merge_drop.copy()

dem_ex_merge_tenure_grouped.loc[:, 'tenure_group'] = pd.cut(dem_ex_merge_tenure_grouped['clnt_tenure_yr'], bins=tenure_bins, labels=tenure_labels, right=False)

tenure_counts_per_test = dem_ex_merge_tenure_grouped.groupby('tenure_group')['Variation'].value_counts()
tenure_counts_per_test_df = tenure_counts_per_test.reset_index(name='count')
print(tenure_counts_per_test_df)

   tenure_group Variation  count
0           0-5      Test   2033
1           0-5   Control   1746
2          6-10      Test  10156
3          6-10   Control   8713
4         11-15      Test   6324
5         11-15   Control   5564
6         16-20      Test   4190
7         16-20   Control   3685
8         21-25      Test   2769
9         21-25   Control   2491
10          26+      Test   1489
11          26+   Control   1328


In [94]:
csv_file_path = 'tenure_group.csv'
tenure_counts_per_test_df.to_csv(csv_file_path, index=False)  # Set index=False to exclude row numbers in the CSV file

print("DataFrame saved to CSV file:", csv_file_path)

DataFrame saved to CSV file: tenure_group.csv


In [54]:
new_age_bins = [0, 30, 50, 70, 90, float('inf')]
new_age_labels = ['0-29', '30-49', '50-69', '70-89', '90+']
dem_ex_merge_age_grouped = dem_ex_merge_drop.copy()
dem_ex_merge_age_grouped.dropna(subset=['clnt_age'], inplace=True)
dem_ex_merge_age_grouped['age_group'] = pd.cut(dem_ex_merge_age_grouped['clnt_age'], bins=new_age_bins, labels=new_age_labels, right=False)

age_counts_per_test = dem_ex_merge_age_grouped.groupby('age_group')['Variation'].value_counts()
age_counts_per_test_df = age_counts_per_test.reset_index(name='count')
print(age_counts_per_test_df)

  age_group Variation  count
0      0-29      Test   4596
1      0-29   Control   3935
2     30-49      Test   9898
3     30-49   Control   8442
4     50-69      Test  10672
5     50-69   Control   9513
6     70-89      Test   1770
7     70-89   Control   1618
8       90+      Test     25
9       90+   Control     18


In [92]:
csv_file_path = 'age_group.csv'
age_counts_per_test_df.to_csv(csv_file_path, index=False)  # Set index=False to exclude row numbers in the CSV file

print("DataFrame saved to CSV file:", csv_file_path)

DataFrame saved to CSV file: age_group.csv


In [ ]:

start_control = df_final_merged[(df_final_merged['Variation'] == 'Control') & (df_final_merged['process_step'] == 'start')].groupby(['client_id', 'visitor_id', 'visit_id']).size().reset_index(name='count').shape[0]
start_test = df_final_merged[(df_final_merged['Variation'] == 'Test') & (df_final_merged['process_step'] == 'start')].groupby(['client_id', 'visitor_id', 'visit_id']).size().reset_index(name='count').shape[0]

confirm_control = df_final_merged[(df_final_merged['Variation'] == 'Control') & (df_final_merged['process_step'] == 'confirm')].groupby(['client_id', 'visitor_id', 'visit_id']).size().reset_index(name='count').shape[0]
confirm_test = df_final_merged[(df_final_merged['Variation'] == 'Test') & (df_final_merged['process_step'] == 'confirm')].groupby(['client_id', 'visitor_id', 'visit_id']).size().reset_index(name='count').shape[0]

# Calculate completion rate
control_completion_rate = round(confirm_control / start_control, 2)
test_completion_rate = round(confirm_test / start_test, 2)

# Print completion rates
print("The control completion rate is:", control_completion_rate)
print("The test completion rate is:", test_completion_rate)